In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers

In [2]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def generate_essay(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    
    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    generated_ids = model.generate(
        model_inputs,
        max_new_tokens = 7500,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id
        
    )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

prompt = '''
Obituary Details: 
Yulonda “Lonnie” Bascia Hollis, 49, of Packard Ave., Kannapolis, N.C., passed away on Tuesday, August 31, 2021 at her residence.  She was born August 18, 1972 in Cabarrus County to Kenneth Crawford and Lorene Hollis Russell. Yulonda was educated in the Cabarrus County School System  and attended A.L. Brown High School.  She was formerly employed with FedEx.  Services will be held on Wednesday, September 8, 2021, at Living Water Church of God 166 N. Little Texas Rd. Kannapolis.  Visitation will be 1:00 PM – 2:00 PM; with funeral services at 2:00 PM. Burial will follow at Bethel Baptist Church Cemetery. She was preceded in death by a brother, Cedric Hill; grandmother, Essie Hollis and stepfathers, Robin Brown  and Cleatus Gill. She leaves to cherish fond memories two daughters, Kiana Hollis and Jasmine Hollis both of Kannapolis; one son, Clinton Thompson of Kannapolis; seven grandchildren; her parents, Kenneth Crawford (Regina) pf Mooresville, NC, Lorene Russell of Kannapolis;  fiancé, Len Harold; one brother, Rodricko Hill (Lavita) of Charlotte, NC; four sisters, Gytanna Hollis of Concord, Shakitta Hill (Jermaine Smith) of Kannapolis, Tara Crawford of Mooresville, NC, Melonie Patrice Love of Dallas, TX; grandmother, Lula Mae Brown of Charlotte, NC; grandfather, Cletus Gill (Sue) of Kannapolis; stepbrother, Rev. Chavis Maxwell (Tosha) of Kannapolis; stepsister, Tameca Robinson of Salisbury and a host of nieces, nephews, cousins and friends.

Task:
Extract the below attributes if exists from the above obituary details.
name, age, death_city, death_state, death_date, birth_date, birth_city, mother, father, grandmother, grandfather, sons, daughters ,spouse, siblings, other_relatives, college, occupation, funeral_home_name, funeral_home_address

Output: The output should be a python dictionary
'''


In [4]:
%%time

text = generate_essay(prompt)
text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CPU times: user 29.9 s, sys: 1.62 s, total: 31.5 s
Wall time: 36.6 s


' {\n"name": "Yulonda "Lonnie" Bascia Hollis",\n"age": 49,\n"death\\_city": "Kannapolis, N.C.",\n"death\\_state": "North Carolina",\n"death\\_date": "August 31, 2021",\n"birth\\_date": "August 18, 1972",\n"birth\\_city": "Cabbartus County",\n"mother": "Lorene Hollis Russell",\n"father": "Kenneth Crawford",\n"grandmother": "Essie Hollis",\n"grandfather": "Cletus Gill",\n"sons": ["Clinton Thompson"],\n"daughters": ["Kiana Hollis", "Jasmine Hollis"],\n"spouse": "Len Harold",\n"siblings": ["Rodricko Hill", "Shakitta Hill", "Tara Crawford", "Melonie Patrice Love", "Rev. Chavis Maxwell", "Tameca Robinson"],\n"college": "",\n"occupation": "",\n"funeral\\_home\\_name": "",\n"funeral\\_home\\_address": ""\n}'